In [74]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor,AdaBoostRegressor
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import lightgbm as LGB
from sklearn.metrics import mean_absolute_error as mae,mean_squared_log_error as rmsle,mean_squared_error as mse
import os,re
import warnings
warnings.filterwarnings('ignore')
PATH='Restaurant prices 23.04.19/Participants_Data_Final'

In [75]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from mlens.ensemble import SuperLearner


In [76]:
import jupyterthemes as jt
from jupyterthemes import jtplot
jt.install_theme('chesterish')
jtplot.style()
jt.get_themes()

['chesterish',
 'grade3',
 'gruvboxd',
 'gruvboxl',
 'monokai',
 'oceans16',
 'onedork',
 'solarizedd',
 'solarizedl']

In [77]:
from concurrent.futures import ThreadPoolExecutor
def parallel_trees(m, fn, n_jobs=8):
        return list(ThreadPoolExecutor(n_jobs).map(fn, m.estimators_))
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)
def get_preds(t): return t.predict(valX)
# %time preds = np.stack(parallel_trees(model, get_preds))
# np.mean(preds[:,0]), np.std(preds[:,0])


In [5]:
print(f'{PATH}'+'Data')
df_raw_1=pd.read_excel(PATH+'/Data_Train.xlsx',
                     dtype={
#                          'TITLE', 
                            'RESTAURANT_ID':np.int32,
#                             'CUISINES', 'TIME', 'CITY', 'LOCALITY',
#                             'RATING':float,
#                             'VOTES',
                            'COST':np.int32
                           })

Restaurant prices 23.04.19/Participants_Data_FinalData


In [81]:
df_raw=pd.read_excel(PATH+'/Data_Train.xlsx',
                     dtype={
#                          'TITLE', 
                            'RESTAURANT_ID':np.int32,
#                             'CUISINES', 'TIME', 'CITY', 'LOCALITY',
#                             'RATING':float,
#                             'VOTES',
                            'COST':np.int32
                           })
df_test=pd.read_excel(PATH+'/Data_Test.xlsx',
                      dtype={
#                          'TITLE', 
                            'RESTAURANT_ID':np.int32,
#                             'CUISINES', 'TIME', 'CITY', 'LOCALITY',
#                             'RATING':float,
#                             'VOTES',
                           })

In [82]:
from pandas.api.types import is_string_dtype
df_temp=df_raw.copy()
Y1=df_temp.COST
df_temp.drop('COST',axis=1,inplace=True)
df_temp=pd.concat([df_temp,df_test])

In [8]:
df_temp.columns

Index(['TITLE', 'RESTAURANT_ID', 'CUISINES', 'TIME', 'CITY', 'LOCALITY',
       'RATING', 'VOTES'],
      dtype='object')

In [83]:
df_temp.TIME.head()

0    11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)
1                      6pm – 11pm (Mon-Sun)
2       11am – 3:30pm, 7pm – 11pm (Mon-Sun)
3                   11:30am – 1am (Mon-Sun)
4                      11am – 1am (Mon-Sun)
Name: TIME, dtype: object

In [ ]:
ss='12midnight – 3am, 11:15am – 12midnight (Mon...'
ss='1pm – 11pm (Mon, Tue, Thu), 1pm – 11:1pm (Wed...'
ss.replace('midnight','am')
ss.replace('noon','pm')
# ss.replace('...',')') handle ...
rad=[]
rad.append('lays')
rad.append('None')
rad.pop()
rad.append('yo')

In [ ]:
# closed=[]
# opened=[]
# for str in df_temp.TIME:
#     str=str.upper()
#     str=str.replace('MIDNIGHT','AM')
#     str=str.replace('NOON','PM')
#     str=str.replace('...',')')
#     str=str.replace('MON','1')
#     str=str.replace('TUE','2')
#     str=str.replace('WED','3')
#     str=str.replace('THU','4')
#     str=str.replace('FRI','5')
#     str=str.replace('SAT','6')
#     str=str.replace('SUN','7')
#     time_new=''
#     j=0
#     closed.append('NONE')
#     opened.append('NONE')
#     for split_1 in str.split(','):
#         split_1=split_1.strip()
#         temp=re.findall("\((.*?)\)",split_1)
#         for item in temp:
#             if('CLOSED' in split_1):
#                 closed.pop()
#                 closed.append(item) 
#             else:
#                 opened.pop()
#                 opened.append(item)
#             if(j>0):time_new+=','
#             time_new+=item
#             j+=1
        
# #         split_1=re.sub("\((.*?)\)",'',split_1)
# #         for split_2 in split_1.split('–'):
# #             split_2=split_2.strip()
# #         print(split_2)
# #     print(split_1)
# #         if(len(temp))>0:
# #             print((temp))
# #             print(split_1)
# # opened

In [ ]:
# days=pd.DataFrame({'Open':opened,'Closed':closed})
# item=days.Open.mode()
# days.Closed.replace('NONE','0',inplace=True)
# days.Open.replace('NONE',item[0],inplace=True)
# # days
# days.loc[(days.Open!=item[0])&(days.Closed=='0')]
# # days.loc[days.Open == '[MON-SUN]']
# # df_temp['Closed']=days.Closed
# # df_temp['Open']=days.Open

In [85]:
# Data PreProcessing

title_lc=LabelEncoder()
cuisine_lc=LabelEncoder()
time_lc=LabelEncoder()
city_lc=LabelEncoder()
locality_lc=LabelEncoder()


# 'TIME'
time_list=[]
times=list(df_temp.TIME)
for i in range (0,len(times)):
    time=times[i]
    time_new=''
    j=0
    for split_1 in time.split(','):
        split_1=split_1.strip().upper()
        temp=re.findall("\((.*?)\)",split_1)
        for item in temp:
            if(j>0):time_new+=','
            time_new+=item
            j+=1
            time_list.append(item)
        split_1=re.sub("\((.*?)\)",'',split_1)
        for split_2 in split_1.split('–'):
            split_2=split_2.strip()
    if(len(time_new)==0):time_new='None'
    times[i]=time_new
df_temp.TIME=times 
time_list.append('None')
time_list=np.unique(np.array(time_list)) 
time_lc.fit(time_list)
print(len(time_list))
del times

#             print(split_2)
#     print(split_1)
#     print((temp))


# 'TITLE'

#Find title-cost connection using groupBy plot
t_arr1=['PAAN SHOP','MESS','BHOJANALAYA','MEAT SHOP','KIOSK',
          'IRANI CAFE','CONFECTIONERY']
t_arr2=['FOOD COURT','BAKERY','NONE','SWEET SHOP','BEVERAGE SHOP']
t_arr3=['None','QUICK BITES','DESSERT PARLOR','FOOD TRUCK']
t_arr4=['CAFÉ','DHABA','CASUAL DINING']
t_arr5=['PUB','BAR','LOUNGE','MICROBREWERY']
t_arr6=['CLUB', 'COCKTAIL BAR',]
t_arr7=['FINE DINING']

title_list=[]
title_band=[]
titles=list(df_temp.TITLE)
for i in range(0,len(titles)):
    title=titles[i]
    title_new=''
    j=0
    for item in title.split(','):
        item=item.strip().upper()
        title_list.append(item)
        if(j>0):title_new+=','
        title_new+=item
        temp_band=0
        if item in t_arr1:temp_band=0
        elif item in t_arr2:temp_band=1
        elif item in t_arr3:temp_band=2   
        elif item in t_arr4:temp_band=3   
        elif item in t_arr5:temp_band=4
        elif item in t_arr6:temp_band=5
        elif item in t_arr7:temp_band=6
        if(len(title_band)==i):title_band.append(temp_band)
        else:
            if(title_band[i]<temp_band):title_band[i]=temp_band
        
        j+=1
    titles[i]=title_new
df_temp.TITLE=titles
df_temp['T_BAND']=title_band
title_list.append('None')
title_list=np.unique(np.array(title_list))  
title_lc.fit(title_list)
print(len(title_list))
del titles

# 'CUISINES'

# df_temp3=df_temp.copy()
# df_temp3['COST']=df_raw.COST
# col='CUISINES_1'
# dd=df_temp3[[col,'COST']].groupby([col]).mean().sort_values(by='COST',ascending=True)
# dd['BAND']=pd.cut(dd.COST,10)
# dict=dd[['BAND']].groupby(['BAND']).groups
# list_values = [ v for v in dict.values() ]
# del df_temp3,dd

#Find cuisine-cost connection using groupBy plot
t_arr1=['PAAN', 'AFGHAN', 'TEA', 'CAFE FOOD', 'BUBBLE TEA', 'STREET FOOD',
       'JUICES', 'CHARCOAL CHICKEN', 'ASSAMESE', 'MITHAI', 'SANDWICH',
       'BURMESE', 'ICE CREAM', 'ORIYA', 'MOMOS', 'ROAST CHICKEN', 'FAST FOOD',
       'WRAPS', 'BEVERAGES', 'DESSERTS', 'ROLLS', 'KEBAB', 'BAKERY',
       'MAHARASHTRIAN', 'SOUTH INDIAN', 'COFFEE', 'BIHARI', 'BOHRI',
       'COFFEE AND TEA' ,'FUSION','GRILL','HAWAIIAN','MISHTI' ,'MONGOLIAN','PANINI']
t_arr2=['BIRYANI', 'KERALA', 'INDONESIAN', 'TAMIL', 'RAW MEATS', 'LUCKNOWI',
       'MIDDLE EASTERN', 'CANTONESE', 'HOT DOGS', 'PAKISTANI', 'TIBETAN',
       'ARABIAN', 'LEBANESE', 'PIZZA', 'BURGER', 'CHETTINAD', 'NORTH EASTERN',
       'IRANIAN', 'TURKISH', 'HEALTHY FOOD', 'MULTI CUISINE', 'BENGALI',
       'ANDHRA', 'SALAD', 'RAJASTHANI', 'AFRICAN', 'MUGHLAI', 'CHINESE',
       'MANGALOREAN', 'INDIAN', 'CAFE', 'SINDHI', 'NORTH INDIAN', 'MALWANI',
        'CUISINE VARIES','GREEK','HOT POT','JEWISH','None','VEGAN']
t_arr3=['AWADHI', 'PARSI', 'BBQ', 'SRI LANKAN', 'NEPALESE', 'NAGA', 'GUJARATI',
       'MEXICAN', 'ETHIOPIAN', 'SEAFOOD', 'SUSHI', 'HYDERABADI', 'GERMAN',
       'AMERICAN','FISH AND CHIPS','SATAY']
t_arr4=['GOAN', 'ITALIAN', 'ASIAN', 'CONTINENTAL', 'KOREAN', 'KONKAN',
       'MALAYSIAN', 'STEAK', 'TEX-MEX', 'BRITISH', 'THAI', 'SINGAPOREAN',
       'PORTUGUESE', 'KASHMIRI', 'RUSSIAN','DUMPLINGS','PAN ASIAN']
t_arr5=['BAR FOOD', 'VIETNAMESE', 'FINGER FOOD', 'MODERN INDIAN','FALAFEL']
t_arr6=['SOUTH AMERICAN', 'MEDITERRANEAN', 'FRENCH', 'EUROPEAN','BELGIAN','POKÉ']
t_arr7=['','JAPANESE', 'DRINKS ONLY', 'ARMENIAN','MODERN AUSTRALIAN']
# t_arr8=['SPANISH']
# t_arr9=['IRISH','MOROCCAN']
# t_arr10=['BRAZILIAN', 'EGYPTIAN','ORIENTAL']
# t_arr11=['PERUVIAN']
t_arr8=['SPANISH','IRISH','MOROCCAN','BRAZILIAN', 'EGYPTIAN','ORIENTAL','PERUVIAN']





cuisine_list=[]
cuisine_band=[]
cuisines=list(df_temp.CUISINES)
for i in range(0,len(cuisines)):
    cuisine=cuisines[i]
    cuisine_new=''
    j=0
    for item in cuisine.split(','):
        item=item.strip().upper()
        cuisine_list.append(item)
        if(j>0):cuisine_new+=','
        cuisine_new+=item
        
        k=0
        temp_band=0
        if item in t_arr1:temp_band=0
        elif item in t_arr2:temp_band=1
        elif item in t_arr3:temp_band=2   
        elif item in t_arr4:temp_band=3   
        elif item in t_arr5:temp_band=4
        elif item in t_arr6:temp_band=5
        elif item in t_arr7:temp_band=6
        elif item in t_arr8:temp_band=7
#         elif item in t_arr9:temp_band=8
#         elif item in t_arr10:temp_band=9
#         elif item in t_arr11:temp_band=10
        if(len(cuisine_band)==i):cuisine_band.append(temp_band)
        else:
            if(cuisine_band[i]<temp_band):cuisine_band[i]=temp_band
        
        j+=1
    cuisines[i]=cuisine_new
df_temp.CUISINES=cuisines
df_temp['C_BAND']=cuisine_band
cuisine_list.append('None')
cuisine_list=np.unique(np.array(cuisine_list)) 
cuisine_lc.fit(cuisine_list)
print(len(cuisine_list))
del cuisines

# 'CITY'
cities=list(df_temp.CITY)
for i in range(0,len(cities)):
    city=cities[i]
    try:
        cities[i]=city.strip().upper()
    except:
        cities[i]='None'

df_temp.CITY=cities
cities=np.unique(np.array(cities))
print(len(cities))
# del cities
city_lc.fit(cities)
# 'LOCALITY'
localities=list(df_temp.LOCALITY)
for i in range(0,len(localities)):
    try:
        localities[i]=(localities[i].strip().upper())
    except:
        localities[i]='NONE'

df_temp.LOCALITY=localities
localities=np.unique(np.array(localities))  
locality_lc.fit(localities)
print(len(localities))
# del localities

# 'RATING'
# Rating -1/0??
df_temp.RATING.fillna(0,inplace=True)
if 'NEW' in df_temp.RATING.values:
    df_temp.RATING.replace('NEW',-1,inplace=True)
if '-' in df_temp.RATING.values:
    df_temp.RATING.replace('-',0,inplace=True)  
if df_temp.RATING.isnull().any():    
    df_temp.RATING.fillna(0,inplace=True)
if is_string_dtype(df_temp.RATING):
    df_temp.RATING=df_temp.RATING.astype('float')

# 'VOTES'
# -1/0??
df_temp.VOTES.fillna(('-1 votes'),inplace=True)
if not (type(df_temp.VOTES.values[0]) == np.int64 ):
    df_temp.VOTES=df_temp.VOTES.apply(lambda x:int(x.split(' ')[0]))
df_temp.head()

1
26
131
434
1588


,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,T_BAND,C_BAND
0,CASUAL DINING,9438,"MALWANI,GOAN,NORTH INDIAN",None,THANE,DOMBIVLI EAST,3.6,49,3,3
1,"CASUAL DINING,BAR",13198,"ASIAN,MODERN INDIAN,JAPANESE",None,CHENNAI,RAMAPURAM,4.2,30,4,6
2,CASUAL DINING,10915,"NORTH INDIAN,CHINESE,BIRYANI,HYDERABADI",None,CHENNAI,SALIGRAMAM,3.8,221,3,2
3,QUICK BITES,6346,"TIBETAN,CHINESE",None,MUMBAI,BANDRA WEST,4.1,24,2,1
4,DESSERT PARLOR,15387,DESSERTS,None,MUMBAI,LOWER PAREL,3.8,165,2,0


In [86]:
# # unsorted lists
  
if 'TITLE' in df_temp.columns.values :
    new=df_temp.TITLE.str.split(',',expand=True)
    for n in new:
        if(new[n].isnull().any()):
            new[n].fillna('None',inplace=True)
        df_temp['TITLE_'+f'{(n+1)}']=new[n]
    df_temp.drop('TITLE',axis=1,inplace=True)  

if 'CUISINES' in df_temp.columns.values :
    new=df_temp.CUISINES.str.split(',',expand=True)
    for n in new:
        if(new[n].isnull().any()):
            new[n].fillna('None',inplace=True)
        df_temp['CUISINES_'+f'{(n+1)}']=new[n]
    df_temp.drop('CUISINES',axis=1,inplace=True)  
    
if 'TIME' in df_temp.columns.values :
    df_temp.TIME.replace('None',df_temp.TIME.mode,inplace=True)
    new=df_temp.TIME.str.split(',',expand=True)
    for n in new:
        if(new[n].isnull().any()):
            new[n].fillna('None',inplace=True)
        df_temp['TIME_'+f'{(n+1)}']=new[n]
    df_temp.drop('TIME',axis=1,inplace=True)    


In [ ]:
# #C_BAND



# df_temp3=df_temp.copy()
# df_temp3['COST']=df_raw.COST
# col='CUISINES_1'
# len(cuisine_list)
# dd=df_temp3[[col,'COST']].groupby([col]).mean().sort_values(by='COST',ascending=True)
# dd['BAND']=pd.cut(dd.COST,10)
# dict=dd[['BAND']].groupby(['BAND']).groups
# list_values = [ v for v in dict.values() ]
# # del df_temp3,dd
# dd[['BAND']].groupby(['BAND']).count()



In [ ]:
df_temp3=df_temp.copy()
df_temp3['COST']=df_raw.COST
# df_temp3['Titleband']=pd.cut(df_temp3.TITLE_1,10)
df_temp3[['T_BAND','COST']].groupby(['T_BAND']).mean().sort_values(by='COST',ascending=True)

In [87]:
# Label Encoding
for col in df_temp.columns.values:
    if 'TITLE' in col:
        df_temp[col]=title_lc.transform(df_temp[col])
    if 'CUISINE' in col:
        df_temp[col]=cuisine_lc.transform(df_temp[col])
    if 'TIME' in col:
        df_temp[col]=time_lc.transform(df_temp[col])    
    if 'CITY' in col:
        df_temp[col]=city_lc.transform(df_temp[col])
    if 'LOCALITY' in col:
        df_temp[col]=locality_lc.transform(df_temp[col])
        

In [88]:

df_temp.drop([
#     'TIME',
              'RESTAURANT_ID'],axis=1,inplace=True)

In [89]:
df_temp.head()

,CITY,LOCALITY,RATING,VOTES,T_BAND,C_BAND,TITLE_1,TITLE_2,CUISINES_1,CUISINES_2,CUISINES_3,CUISINES_4,CUISINES_5,CUISINES_6,CUISINES_7,CUISINES_8,TIME_1
0,397,320,3.6,49,3,3,5,21,74,47,91,92,92,92,92,92,0
1,71,1188,4.2,30,4,6,5,1,7,82,62,92,92,92,92,92,0
2,71,1242,3.8,221,3,2,5,21,91,29,17,55,92,92,92,92,0
3,271,148,4.1,24,2,1,24,21,126,29,92,92,92,92,92,92,0
4,271,688,3.8,165,2,0,9,21,34,92,92,92,92,92,92,92,0


In [90]:
def print_score(model,check=True,file='1'):
#     preds=sc.inverse_transform(model.predict(valX))
#     preds=preds.astype(int)
    
#     
    
#     sc.inverse_transform
    if check:
        print('Score '+f'{model.score(valX,valY)}')
        preds=model.predict(valX)
#         print(pd.DataFrame(preds))
#         print('rmsle '+f'{1-rmsle(preds,valY)}')
        print('mse '+f'{sqrt(mse(preds,valY))}')
    else:
        preds=model.predict(df_test)
        preds = pd.DataFrame(preds, columns = ['COST']) # Converting to dataframe
        preds.to_excel(PATH+'/RFR'+file+'.xlsx', index = False ) # Saving the output in to an excel
        print(preds)
    # median prediction
#     preds[:len(preds)]=df_raw['COST'].median()

#     print('Correct: '+f'{(preds==valY).sum()}'+' Out of '+f'{len(preds)}')

In [98]:
from math import log,sqrt
# One Hot Encoding

str=[]

df_temp2=df_temp.copy()

# drop=['CUISINES_4','CUISINES_5','CUISINES_6','CUISINES_7','CUISINES_8']

# df_temp2=df_temp2.drop(str,axis=1)

# df_temp2=df_temp.copy()

# df_temp2.drop(['Closed'],axis=1,inplace=True)

for col in df_temp2.columns.values:
#     if 'T_BAND' in col:
#         str.append(col)
    if 'TITLE' in col:
        str.append(col)
    if 'CUISINE' in col:
        str.append(col)
    if 'CITY' in col:
        str.append(col)
    if 'LOCALITY' in col:
        str.append(col)
    if 'TIME' in col:
        str.append(col) 
#     if 'Closed' in col:
#         str.append(col)
#     if 'Open' in col:
#         str.append(col)

df_temp2 = pd.get_dummies(df_temp2,columns=str)   # gives (1451, 221) 
#comment when run w/out FI

df_temp2=df_temp2[cols]


df_raw=df_temp2[:df_raw.shape[0]]

df_test=df_temp2[df_raw.shape[0]:]

df_raw['COST']=Y1

df_temp2=df_raw.copy()

X=df_temp2.drop([
    'COST',
#                  'LOCALITY','CITY'
                ],axis=1)


Y=df_temp2.COST
itr=0
while(True):
    trainX,valX,trainY,valY=train_test_split(X,Y)
    trainX.shape,trainY.shape,valX.shape,valY.shape

# model=GradientBoostingRegressor(random_state=42,loss = 'huber',n_estimators=1000, max_depth=4,subsample=0.97,max_leaf_nodes=50,verbose=False)
    model=LGB.LGBMRegressor(learning_rate=0.1,num_leaves=100,max_depth=6,bin=500,min_child_samples=10)    
    model.fit(trainX,trainY)
    score=model.score(valX,valY)
    print(score)
    if(score>=0.79):break
    if(itr>=25):break
    itr+=1

0.7258244397174526
0.746775149266887
0.7795056949385477
0.7062538181727444
0.6967908889951588
0.6906141879869412
0.7514661311642197
0.794863239486891


In [97]:
feat=rf_feat_importance(model,X)
cols=feat[feat.imp>0].cols.values
"""
GBR
No Dummies 0.76,
No FI; Dummy Title; train 0.7583,test 0.8371

"""
feat.shape,cols.shape
# feat

((171, 2), (136,))

In [94]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Dropout,Input
from keras.layers.merge import concatenate
from keras.models import load_model,Model
from numpy import argmax
from os import makedirs


In [99]:
NN_model = Sequential()
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# NN_model.add(Dropout(rate=0.8))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
# NN_model.summary()
checkpoint_name = 'Weights---{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]
NN_model.fit(X, Y, epochs=50, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Train on 10152 samples, validate on 2538 samples
Epoch 1/50
10152/10152 [==============================] - 12s 1ms/step - loss: 332.2279 - mean_absolute_error: 332.2279 - val_loss: 233.4604 - val_mean_absolute_error: 233.4604

Epoch 00001: val_loss improved from inf to 233.46036, saving model to Weights---233.46036.hdf5
Epoch 2/50
10152/10152 [==============================] - 5s 494us/step - loss: 230.9571 - mean_absolute_error: 230.9571 - val_loss: 210.9057 - val_mean_absolute_error: 210.9057ss: 233.9015 - mean_absolute_er

Epoch 00002: val_loss improved from 233.46036 to 210.90568, saving model to Weights---210.90568.hdf5
Epoch 3/50
10152/10152 [==============================] - 5s 493us/step - loss: 218.1561 - mean_absolute_error: 218.1561 - val_loss: 205.1393 - val_mean_absolute_error: 205.1393

Epoch 00003: val_loss improved from 210.90568 to 205.13933, saving model to Weights---205.13933.hdf5
Epoch 4/50
10152/10152 [==============================] - 5s 494us/step - loss: 214.575

10152/10152 [==============================] - 5s 472us/step - loss: 176.7437 - mean_absolute_error: 176.7437 - val_loss: 181.1028 - val_mean_absolute_error: 181.1028

Epoch 00034: val_loss did not improve from 173.95034
Epoch 35/50
10152/10152 [==============================] - 5s 474us/step - loss: 171.6435 - mean_absolute_error: 171.6435 - val_loss: 208.8851 - val_mean_absolute_error: 208.8851

Epoch 00035: val_loss did not improve from 173.95034
Epoch 36/50
10152/10152 [==============================] - 5s 479us/step - loss: 176.5591 - mean_absolute_error: 176.5591 - val_loss: 185.2827 - val_mean_absolute_error: 185.2827

Epoch 00036: val_loss did not improve from 173.95034
Epoch 37/50
10152/10152 [==============================] - 5s 468us/step - loss: 171.1774 - mean_absolute_error: 171.1774 - val_loss: 181.4238 - val_mean_absolute_error: 181.4238

Epoch 00037: val_loss did not improve from 173.95034
Epoch 38/50
10152/10152 [==============================] - 5s 465us/step - loss:

In [96]:
# Load wights file of the best model :
wights_file = 'Weights---173.31224.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
# preds = NN_model.predict(X)
# mae(preds,valY)
print_score(NN_model,False)

             COST
0      814.164124
1      349.325378
2      688.905273
3      391.558868
4      276.365356
5      356.984619
6      671.946045
7      645.475647
8      441.057983
9      475.297241
10     256.615875
11     315.942627
12    1760.119019
13    1603.342285
14     785.747070
15     944.969238
16     379.316589
17     385.561768
18     220.049957
19     209.563583
20     592.941223
21     501.280823
22     937.047852
23     354.173950
24     291.399414
25     299.044861
26    1117.044678
27     558.775696
28     431.041199
29     398.097656
...           ...
4201   275.105652
4202   332.575806
4203   412.571777
4204   273.909485
4205   301.760498
4206  3301.350830
4207   308.287262
4208   602.404663
4209   623.616333
4210   610.369812
4211   385.733551
4212   784.811462
4213   563.450623
4214   360.783752
4215   564.270203
4216   380.724854
4217  2103.796143
4218   272.057556
4219   598.009094
4220   501.454620
4221   736.645569
4222   420.858459
4223   619.833313
4224   344

In [71]:
# preds = pd.DataFrame(yhat, columns = ['COST']) # Converting to dataframe
# preds.to_excel(PATH+'/RFR1.xlsx', index = False ) # Saving the output in to an excel
# print(preds)